# Preparation for Building CNN Model: Define Supporting Functions

### Initialize Wegihts in Filter

In [24]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

def initialize_weights (filter_shape):
    inti_random = tf.truncated_normal(filter_shape, stddev = 0.1)
    return (tf.Variable(inti_random))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Initialize Bias

In [25]:
def initialize_bias (bias_shape):
    inti_bias = tf.constant(0.1, shape=bias_shape)
    return (tf.Variable(inti_bias))

### Set Up Convolutional Layer and Perform Convolution: Dot Product(x *W)

In [26]:
def create_convolutional_layer_and_compute_dot(inputs, filter_shape):
    filter_inti_with_weight = initialize_weights(filter_shape)
    conv_layer_out = tf.nn.conv2d(inputs, filter_inti_with_weight, strides=[1,1,1,1], padding = 'SAME')
    return(conv_layer_out)

### Set Up Convolutional Layer and Perform Convolution: Dot Product + bias (x *W+b)

In [27]:
#inputs = output from convolution_layer
#filter_shape = the elements number of the convolution layer which has 4 elements, so [3]
def create_relu(inputs, filter_shape):
    b = initialize_bias([filter_shape[3]])
    relu_layer = tf.nn.relu(inputs+b)
    return(relu_layer)


### Set Up pooling Layer and reduce spatial size

In [28]:
def create_maxpooling_reduce(inputs):
    pooling_layer_outputs = tf.nn.max_pool(inputs, ksize=[1,2,2,1], strides=[1,2,2,1], padding = 'SAME')
    return (pooling_layer_outputs)

### Set Up Fully Connected Layer and Perform Computation: (inputs*Weight)+Bias

In [29]:
def create_fully_connect_layer_and_compute_dotproduct_plus_bias(inputs,output_size):
    input_size = int(inputs.get_shape()[1])
    W = initialize_weights([input_size,output_size])
    b = initialize_bias ([output_size])
    fc = tf.matmul(inputs,W)+b
    return(fc)

 ### Phase I : Build the Convolution Neural Network

In [30]:
x = tf.placeholder(tf.float32, shape=[None,784])
y_true = tf.placeholder(tf.float32,[None,10])

### Reshape the Input Placeholder x

In [31]:
"""
1 is gray scale
-1 is reshape to be the same scale as the old one
"""
x_image = tf.reshape(x,[-1,28,28,1])


### Create 1st Convolution Layer, ReLU Layer, and Perform Computational: x*W+b

In [32]:
conv_layer_1_out \
    = create_convolutional_layer_and_compute_dot(x_image, filter_shape = [4,4,1,32])

relu_layer_1 \
    =create_relu(conv_layer_1_out, filter_shape = [4,4,1,32])

### Create 1st Pooling Layer and Reduce Spatial Size

In [33]:
pooling_layer_1_outputs = create_maxpooling_reduce(relu_layer_1)

### Create 2nd Convolution Layer, ReLU Layer, and Perform Computational: x*W+b

In [34]:
conv_layer_2_out \
    = create_convolutional_layer_and_compute_dot(pooling_layer_1_outputs, filter_shape = [4,4,32,64])

relu_layer_2 \
    =create_relu(conv_layer_2_out, filter_shape = [4,4,32,64])

### Create 2nd Pooling Layer and Reduce Spatial Size

In [35]:
pooling_layer_2_outputs = create_maxpooling_reduce(relu_layer_2)

### Reshape/Flatten Data Making It Ready to be Fed Into 1st FC Layer

In [36]:
pooling_layer_2_outputs_flat =  tf.reshape(pooling_layer_2_outputs, [-1,7 * 7 * 64])

### Create 1st FC Layer,ReLU Layer, and Output Data to Dropout Layer

In [37]:
fc_layer_1_outputs = create_fully_connect_layer_and_compute_dotproduct_plus_bias(pooling_layer_2_outputs_flat, output_size = 1024)
fc_relu_1=tf.nn.relu(fc_layer_1_outputs)

### Create Dropout Layer and Dropout a Fraction of Outputs Randomly

In [38]:
hold_prob = tf.placeholder(tf.float32)
fc_dropout = tf.nn.dropout(fc_relu_1,keep_prob=hold_prob)

### Create Final FC Layer, Compute (xW+b), and Product Final Output

In [39]:
Y_pred = create_fully_connect_layer_and_compute_dotproduct_plus_bias(fc_dropout, output_size=10)

### Define Loss Function and Calculate Softmax Cross Entropy Loss

In [40]:
softmax_corss_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits = Y_pred)
cross_entropy_mean = tf.reduce_mean(softmax_corss_entropy)

### Create an Optimize CNN Model and Set Learning Rate

In [41]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)

### Create a Trainer to train CNN Model

In [42]:
cnn_trainer = optimizer.minimize(cross_entropy_mean)

## Train and Test CNN Deep learning model on MNIST Dataset

### Initialize All Variable

In [43]:
vars_initializer = tf.global_variables_initializer()

### Set the steps

In [44]:
steps = 5000

### Run tf.Session() to Train and Test Deep Learning CNN Model

In [45]:
with tf.Session() as sess:
    sess.run(vars_initializer)
    print('\tON STEP\t\tACCURACY')
    for i in range (steps):
        batch_x, batch_y = mnist.train.next_batch(50)
        sess.run(cnn_trainer, feed_dict={x: batch_x, y_true: batch_y, hold_prob:0.5})
        if i % 100 == 0:
            matches = tf.equal(tf.argmax(Y_pred, 1), tf.argmax(y_true, 1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            test_accuracy = sess.run( acc, feed_dict={x:mnist.test.images, \
                                                      y_true:mnist.test.labels, \
                                                      hold_prob: 1.0} )
            print('\t{}'.format(i), '\t\t', test_accuracy)

	ON STEP		ACCURACY
	0 		 0.1301
	100 		 0.9316
	200 		 0.9609
	300 		 0.9666
	400 		 0.9726
	500 		 0.9714
	600 		 0.9777
	700 		 0.9792
	800 		 0.9815
	900 		 0.9811
	1000 		 0.9827
	1100 		 0.9853
	1200 		 0.9855
	1300 		 0.9855
	1400 		 0.9822
	1500 		 0.9828
	1600 		 0.988
	1700 		 0.9887
	1800 		 0.9862
	1900 		 0.9886
	2000 		 0.9894
	2100 		 0.9893
	2200 		 0.9895
	2300 		 0.9878
	2400 		 0.9893
	2500 		 0.9887
	2600 		 0.9876
	2700 		 0.9906
	2800 		 0.99
	2900 		 0.989
	3000 		 0.9909
	3100 		 0.9891
	3200 		 0.9897
	3300 		 0.9906
	3400 		 0.9928
	3500 		 0.9902
	3600 		 0.9905
	3700 		 0.9906
	3800 		 0.9905
	3900 		 0.9907
	4000 		 0.9909
	4100 		 0.9911
	4200 		 0.9872
	4300 		 0.9915
	4400 		 0.9911
	4500 		 0.99
	4600 		 0.9906
	4700 		 0.9919
	4800 		 0.9919
	4900 		 0.99
